# MinCut Karger + Karger&Stein

- **Author:** *Mariusz Wiśniewski*


## Task

- Analyze the asymptotic complexity of *karger_union_find* and of *karger_stein_union_find* with Google benchmarking library.
- Reimplement/optimize the two *MinCut* procedures with a running time better than the reference code ([GitHub repository](https://github.com/ArthurRouquan/KargerSteinAlgorithm)).

## Notebook setup

In [1]:
!apt install clang-10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang-10 is already the newest version (1:10.0.0-4ubuntu1~18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


### Download the code

In [2]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

fatal: destination path 'benchmark' already exists and is not an empty directory.
fatal: destination path 'benchmark/googletest' already exists and is not an empty directory.


### Organize the code and install

In [3]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.17.1") 
-- git version: v1.6.0-10-g7fad964a normalized to 1.6.0.10
-- Version: 1.6.0.10
-- Performing Test HAVE_CXX_FLAG_STD_CXX11
-- Performing Test HAVE_CXX_FLAG_STD_CXX11 - Success
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WERROR
-- Performing Test HAVE_CXX_FLAG_WERROR - Success
-- Performing Test HAVE_CXX_FLAG_PEDANTIC
-- Performing Test HAVE_CXX_FLAG_

### Get [jngen](https://github.com/ifsmirnov/jngen) library

In [4]:
!wget https://raw.githubusercontent.com/ifsmirnov/jngen/master/jngen.h

--2021-10-18 20:57:05--  https://raw.githubusercontent.com/ifsmirnov/jngen/master/jngen.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182845 (179K) [text/plain]
Saving to: ‘jngen.h.1’

jngen.h.1           100%[===================>] 178.56K  --.-KB/s    in 0.003s  

2021-10-18 20:57:05 (58.1 MB/s) - ‘jngen.h.1’ saved [182845/182845]



## Profiling the Karger and the Karger&Stein mincut algorithm

To calculate the number of edges given the number of vertices and the density of an undirected graph, the following formula was used.
$$D = \frac{|E|}{\binom{|V|}{2}} = \frac{2|E|}{|V|(|V|-1)}$$
By transforming the formula, we obtain
$$|E| = \frac{D|V|(|V|-1)}{2}$$



## Original (unchanged) algorithm

In [5]:
%%writefile original_karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <random>
#include <stack>
#include <vector>

#include "jngen.h"

const float DENSITY = 0.5;

auto& prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
} 


template <typename node_t>
struct Edge { node_t tail, head; };

/* Represents a graph with n vertices as a collection of edges whose vertices are indexed between 0
and n - 1. */
template <typename node_t>
struct EdgesVectorGraph
{
    node_t n; // number of vertices
    std::vector<Edge<node_t>> edges;
};


template<typename node_t>
EdgesVectorGraph<node_t> generateRandomConnectedGraph(int num_of_vertices) {
    int n_edges = static_cast<int>(static_cast<double>(num_of_vertices) *
                                    static_cast<double>(num_of_vertices - 1) * DENSITY / 2.0);
    auto g = Graph::random(num_of_vertices, n_edges).connected().g();

    EdgesVectorGraph<node_t> graph;
    graph.n = num_of_vertices;
    graph.edges.reserve(n_edges);
    for (const auto &e: g.edges()) {
        graph.edges.emplace_back(
                Edge<node_t>({static_cast<node_t>(e.first),
                             static_cast<node_t>(e.second)}));
    }

    return graph;
}


/* An Union-Find data structure (https://fr.wikipedia.org/wiki/Union-Find) is a data structure that
stores a partition of a set into disjoint subsets. Tree height is controlled using union by size.
Use path compression technique. */
template <typename T>
struct UnionFind
{
    struct Subset { T id, size; Subset(T _id, T _size) : id(_id), size(_size) {}};
    std::vector<Subset> subsets;
    T nb_subsets;

    UnionFind(T n) : nb_subsets{n} { 
        subsets.reserve(n);
        for (T i = 0; i < n; ++i)
            subsets.emplace_back(i, 1);
    }

    auto find(T x) {
        auto root = x;
        while (root != subsets[root].id) { root = subsets[root].id; }
        while (subsets[x].id != root) { auto id = subsets[x].id; subsets[x].id = root; x = id; }
        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x); auto const j = find(y);
        if (i == j) return;
        if (subsets[i].size < subsets[j].size) { subsets[i].id = j; subsets[j].size += subsets[i].size; }
        else                                   { subsets[j].id = i; subsets[i].size += subsets[j].size; }
        --nb_subsets;
    }

    bool connected(T x, T y) { return find(x) == find(y); }
};


/* A data structure representing a cut of a graph. */
template <typename node_t>
struct GraphCut
{
    std::size_t cut_size;
    UnionFind<node_t> uf; // used to identify the two partitions of nodes
    GraphCut(std::size_t _cut_size, UnionFind<node_t> _uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const& other) const { return cut_size < other.cut_size; }

    auto get_partitions() const {
        std::vector<node_t> P, Q;
        P.reserve(uf.subsets[0].size); Q.reserve(std::size(uf.subsets) - std::size(P));
        for (node_t i = 0; i < std::size(uf.subsets); ++i)
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        return std::array{P, Q};
    }
};


/* Karger's contraction algorithm in O(n + mα(n)) using an Union-Find data structure to keep track
of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
cut. The graph isn't per se modifed, only its vector of edges is shuffled. */
template <typename node_t>
GraphCut<node_t> karger_union_find(EdgesVectorGraph<node_t>& graph)
{
    auto& mt = prng_engine();
    UnionFind uf{graph.n};
    auto start = begin(graph.edges);
    for (int m = size(graph.edges) - 1; uf.nb_subsets != 2; ++start, --m) {
        std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
        uf.merge(start->tail, start->head);
    }
    return {(std::size_t) std::count_if(start, end(graph.edges), [&](auto e)
        { return !uf.connected(e.tail, e.head); }), std::move(uf)};
}


/* Kargen-Stein's contraction recursive algorithm. Instead of using a straighforward recursion, we
keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high probabili
-ty of obtaining the minimum global cut. */
template <typename node_t>
auto karger_stein_union_find(EdgesVectorGraph<node_t> const& input_graph)
{
    /* A data structure to hold an intermediate contracted graph state. The Union-Find structure
    is used to keep track of the merged nodes. */ 
    struct ContractedGraph : EdgesVectorGraph<node_t> { UnionFind<node_t> uf; };

    /* Contracts the given graph until it has nb_vertices vertices. The graph isn't per se modifed,
    only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph& graph, node_t nb_vertices) {   
        UnionFind uf{graph.uf};
        auto start = begin(graph.edges);
        for (int m = size(graph.edges) - 1; uf.nb_subsets != nb_vertices; ++start, --m) {
            std::iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
            uf.merge(start->tail, start->head);
        }
        decltype(graph.edges) edges;
        edges.reserve(end(graph.edges) - start);
        std::copy_if(start, end(graph.edges), std::back_inserter(edges),
            [&](auto e){ return !uf.connected(e.tail, e.head); }); // remove self-loops
        return ContractedGraph{nb_vertices, std::move(edges), std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph&& graph){ return GraphCut{std::size(graph.edges), graph.uf}; };
    
    double INV_SQRT_2 = 1.0 / std::sqrt(2);
    GraphCut<node_t> best_minimum_cut{input_graph.n, {{}}};
    std::stack<ContractedGraph, std::vector<ContractedGraph>> graphs;
    graphs.push({input_graph.n, input_graph.edges, {input_graph.n}});

    while (!graphs.empty()) // algorithm's main loop
    {
        auto graph = graphs.top();
        graphs.pop();

        if (graph.n <= 6) {
            best_minimum_cut = std::min(best_minimum_cut, cut(contract(graph, 2)));
        } else {
            node_t t = 1 + std::ceil(graph.n * INV_SQRT_2);
            graphs.push(contract(graph, t));
            graphs.push(contract(graph, t));
        }
    }

    return best_minimum_cut;
}

Overwriting original_karger.hpp


In [6]:
%%writefile original_karger_bm.cpp
#include <benchmark/benchmark.h>

#include "original_karger.hpp"

static void BM_Original_Karger(benchmark::State &state) {
    auto n_vertices = static_cast<int>(state.range(0));
    size_t n_edges = 0;
    srand(time(nullptr));

    for (auto _ : state) {
        state.PauseTiming();

        EdgesVectorGraph<uint32_t> graph = generateRandomConnectedGraph<uint32_t>(n_vertices);
        n_edges = graph.edges.size();

        state.ResumeTiming();

        benchmark::DoNotOptimize(karger_union_find(graph));
    }
    state.counters["|V|"] = n_vertices;
    state.counters["|E|"] = static_cast<int>(n_edges);

    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_Original_Karger)
        ->RangeMultiplier(2)
        ->Range(1U << 2U, 1U << 10U)
        ->Complexity();

BENCHMARK_MAIN();


Overwriting original_karger_bm.cpp


In [7]:
%%writefile original_karger_stein_bm.cpp
#include <benchmark/benchmark.h>

#include "original_karger.hpp"

static void BM_Original_Karger_Stein(benchmark::State &state) {
    auto n_vertices = static_cast<int>(state.range(0));
    size_t n_edges = 0;
    srand(time(nullptr));

    for (auto _ : state) {
        state.PauseTiming();

        EdgesVectorGraph<uint32_t> graph = generateRandomConnectedGraph<uint32_t>(n_vertices);
        n_edges = graph.edges.size();

        state.ResumeTiming();

        benchmark::DoNotOptimize(karger_stein_union_find(graph));
    }
    state.counters["|V|"] = n_vertices;
    state.counters["|E|"] = static_cast<int>(n_edges);

    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_Original_Karger_Stein)
        ->RangeMultiplier(2)
        ->Range(1U << 2U, 1U << 10U)
        ->Complexity();

BENCHMARK_MAIN();


Overwriting original_karger_stein_bm.cpp


## Improved iterative algorithm

In [8]:
%%writefile karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <cstdint>
#include <iostream>
#include <random>
#include <stack>
#include <vector>

#include "jngen.h"

const float DENSITY = 0.5;
const int N_VERTICES_THRESHOLD = 6; // Note that 6 is chosen in the algorithm as 6/√2 + 1 >= 2
const double INV_SQRT_2 = 1.0 / sqrt(2);

inline auto &prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
}

template<typename node_t>
struct Edge {
    node_t tail, head;
};

/* Represents a graph with n_vertices as a collection of edges whose vertices are indexed between 0
and n - 1. */
template<typename node_t>
struct EdgesVectorGraph {
    node_t n_vertices;
    std::vector<Edge<node_t>> edges;
};

template<typename node_t>
auto generateRandomConnectedGraph(int n_vertices) -> EdgesVectorGraph<node_t> {
    int n_edges = static_cast<int>(static_cast<double>(n_vertices) *
                                    static_cast<double>(n_vertices - 1) * DENSITY / 2.0);
    auto g = Graph::random(n_vertices, n_edges).connected().g();

    EdgesVectorGraph<node_t> graph;
    graph.n_vertices = n_vertices;
    graph.edges.reserve(n_edges);

    for (const auto &e: g.edges()) {
        graph.edges.emplace_back(
                Edge<node_t>({static_cast<node_t>(e.first),
                              static_cast<node_t>(e.second)}));
    }

    return graph;
}

/* A Union-Find data structure (https://fr.wikipedia.org/wiki/Union-Find) is a data structure that
stores a partition of a set into disjoint subsets. Tree height is controlled using union by size.
Use path compression technique. */
template<typename T>
struct UnionFind {
    struct Subset {
        T id, size;

        Subset(T _id, T _size) : id(_id), size(_size) {}
    };

    std::vector<Subset> subsets;
    T n_subsets;

    UnionFind(T n) : n_subsets{n} {
        subsets.reserve(n);
        for (T i = 0; i < n; ++i) {
            subsets.emplace_back(i, 1);
        }
    }

    auto find(T x) -> T {
        auto root = x;
        while (root != subsets[root].id) {
            root = subsets[root].id;
        }
        while (subsets[x].id != root) {
            auto id = subsets[x].id;
            subsets[x].id = root;
            x = id;
        }
        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x);
        auto const j = find(y);

        if (i == j) {
            return;
        }

        if (subsets[i].size < subsets[j].size) {
            subsets[i].id = j;
            subsets[j].size += subsets[i].size;
        } else {
            subsets[j].id = i;
            subsets[i].size += subsets[j].size;
        }
        --n_subsets;
    }

    auto connected(T x, T y) -> bool {
        return find(x) == find(y);
    }
};

/* A data structure representing a cut of a graph. */
template<typename node_t>
struct GraphCut {
    size_t cut_size;
    UnionFind<node_t> uf;// used to identify the two partitions of nodes
    GraphCut(size_t _cut_size, const UnionFind<node_t> &_uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const &other) const {
        return cut_size < other.cut_size;
    }

    auto get_partitions() const {
        std::vector<node_t> P;
        std::vector<node_t> Q;
        P.reserve(uf.subsets[0].size);
        Q.reserve(size(uf.subsets) - size(P));

        for (node_t i = 0; i < size(uf.subsets); ++i) {
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        }

        return std::array{P, Q};
    }
};

/* Karger's contraction algorithm in O(n + mα(n)) using n Union-Find data structure to keep track
of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
cut. The graph isn't per se modified, only its vector of edges is shuffled. */
template<typename node_t>
auto karger_union_find(EdgesVectorGraph<node_t> &graph) -> GraphCut<node_t> {
    auto &mt = prng_engine();
    UnionFind uf{graph.n_vertices};

    auto start = begin(graph.edges);
    for (int m = size(graph.edges) - 1; uf.n_subsets != 2; ++start, --m) {
        iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
        uf.merge(start->tail, start->head);
    }

    return {
            static_cast<size_t>(count_if(
                    start, end(graph.edges), [&](auto e) {
                        return !uf.connected(e.tail, e.head);
                    })),
            std::move(uf)
    };
}

/* Karger-Stein's contraction recursive algorithm. Instead of using a straightforward recursion, we
keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high probability
 of obtaining the minimum global cut. */
template<typename node_t>
auto karger_stein_union_find(const EdgesVectorGraph<node_t> &input_graph) -> GraphCut<node_t> {
    /* A data structure to hold an intermediate contracted graph state. The Union-Find structure
    is used to keep track of the merged nodes. */
    struct ContractedGraph : EdgesVectorGraph<node_t> {
        UnionFind<node_t> uf;
    };

    /* Contracts the given graph until it has n_vertices vertices. The graph isn't per se modified,
    only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph &graph, node_t n_vertices) -> ContractedGraph {
        UnionFind uf{graph.uf};

        auto start = begin(graph.edges);
        for (int m = size(graph.edges) - 1; uf.n_subsets != n_vertices; ++start, --m) {
            iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
            uf.merge(start->tail, start->head);
        }

        decltype(graph.edges) edges;
        edges.reserve(end(graph.edges) - start);
        auto it = copy_if(start, end(graph.edges), begin(edges),
                          [&](auto e) {
                              return !uf.connected(e.tail, e.head);
                          });// remove self-loops
        edges.erase(it, edges.end());

        return ContractedGraph{n_vertices, std::move(edges), std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph &&graph) -> GraphCut<node_t> {
        return GraphCut{size(graph.edges), graph.uf};
    };

    GraphCut<node_t> best_minimum_cut{input_graph.n_vertices, {{}}};
    std::stack<ContractedGraph, std::vector<ContractedGraph>> graphs;
    graphs.push({input_graph.n_vertices, input_graph.edges, {input_graph.n_vertices}});

    while (!graphs.empty()) {// algorithm's main loop
        auto graph = graphs.top();
        graphs.pop();

        if (graph.n_vertices <= N_VERTICES_THRESHOLD) {
            best_minimum_cut = std::min(best_minimum_cut, cut(contract(graph, 2)));
        } else {
            node_t t = 1 + ceil(graph.n_vertices * INV_SQRT_2);
            graphs.push(contract(graph, t));
            graphs.push(contract(graph, t));
        }
    }

    return best_minimum_cut;
}


Overwriting karger.hpp


In [9]:
%%writefile karger_bm.cpp
#include <benchmark/benchmark.h>

#include "karger.hpp"

static void BM_Karger(benchmark::State &state) {
    auto n_vertices = static_cast<int>(state.range(0));
    size_t n_edges = 0;

    for (auto _: state) {
        state.PauseTiming();

        EdgesVectorGraph<uint32_t> graph = generateRandomConnectedGraph<uint32_t>(n_vertices);
        n_edges = graph.edges.size();

        state.ResumeTiming();

        benchmark::DoNotOptimize(karger_union_find(graph));
    }
    state.counters["|V|"] = n_vertices;
    state.counters["|E|"] = static_cast<int>(n_edges);

    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_Karger)
        ->RangeMultiplier(2)
        ->Range(1U << 2U, 1U << 10U)
        ->Complexity();

BENCHMARK_MAIN();


Overwriting karger_bm.cpp


In [10]:
%%writefile karger_stein_bm.cpp
#include <benchmark/benchmark.h>

#include "karger.hpp"

static void BM_Karger_Stein(benchmark::State &state) {
    auto n_vertices = static_cast<int>(state.range(0));
    size_t n_edges = 0;

    for (auto _: state) {
        state.PauseTiming();

        EdgesVectorGraph<uint32_t> graph = generateRandomConnectedGraph<uint32_t>(n_vertices);
        n_edges = graph.edges.size();

        state.ResumeTiming();

        benchmark::DoNotOptimize(karger_stein_union_find(graph));
    }
    state.counters["|V|"] = n_vertices;
    state.counters["|E|"] = static_cast<int>(n_edges);

    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_Karger_Stein)
        ->RangeMultiplier(2)
        ->Range(1U << 2U, 1U << 10U)
        ->Complexity();

BENCHMARK_MAIN();


Overwriting karger_stein_bm.cpp


## Improved recursive Karger-Stein Min Cut algorithm

In [11]:
%%writefile recursive_karger.hpp
#pragma once

#include <algorithm>
#include <cmath>
#include <cstdint>
#include <iostream>
#include <random>
#include <stack>
#include <vector>

#include "jngen.h"

const float DENSITY = 0.5;
const int N_VERTICES_THRESHOLD = 6; // Note that 6 is chosen in the algorithm as 6/√2 + 1 >= 2
const double INV_SQRT_2 = 1.0 / sqrt(2);

inline auto &prng_engine() {
    thread_local static std::mt19937 engine{std::random_device{}()};
    return engine;
}

template<typename node_t>
struct Edge {
    node_t tail, head;
};

/* Represents a graph with n_vertices as a collection of edges whose vertices are indexed between 0
and n - 1. */
template<typename node_t>
struct EdgesVectorGraph {
    node_t n_vertices;
    std::vector<Edge<node_t>> edges;
};

template<typename node_t>
auto generateRandomConnectedGraph(int n_vertices) -> EdgesVectorGraph<node_t> {
    int n_edges = static_cast<int>(static_cast<double>(n_vertices) *
                                    static_cast<double>(n_vertices - 1) * DENSITY / 2.0);
    auto g = Graph::random(n_vertices, n_edges).connected().g();

    EdgesVectorGraph<node_t> graph;
    graph.n_vertices = n_vertices;
    graph.edges.reserve(n_edges);

    for (const auto &e: g.edges()) {
        graph.edges.emplace_back(
                Edge<node_t>({static_cast<node_t>(e.first),
                              static_cast<node_t>(e.second)}));
    }

    return graph;
}

/* A Union-Find data structure (https://fr.wikipedia.org/wiki/Union-Find) is a data structure that
stores a partition of a set into disjoint subsets. Tree height is controlled using union by size.
Use path compression technique. */
template<typename T>
struct UnionFind {
    struct Subset {
        T id, size;

        Subset(T _id, T _size) : id(_id), size(_size) {}
    };

    std::vector<Subset> subsets;
    T n_subsets;

    UnionFind(T n) : n_subsets{n} {
        subsets.reserve(n);
        for (T i = 0; i < n; ++i) {
            subsets.emplace_back(i, 1);
        }
    }

    auto find(T x) -> T {
        auto root = x;
        while (root != subsets[root].id) {
            root = subsets[root].id;
        }
        while (subsets[x].id != root) {
            auto id = subsets[x].id;
            subsets[x].id = root;
            x = id;
        }
        return root;
    }

    void merge(T x, T y) {
        auto const i = find(x);
        auto const j = find(y);

        if (i == j) {
            return;
        }

        if (subsets[i].size < subsets[j].size) {
            subsets[i].id = j;
            subsets[j].size += subsets[i].size;
        } else {
            subsets[j].id = i;
            subsets[i].size += subsets[j].size;
        }
        --n_subsets;
    }

    bool connected(T x, T y) {
        return find(x) == find(y);
    }
};

/* A data structure representing a cut of a graph. */
template<typename node_t>
struct GraphCut {
    size_t cut_size;
    UnionFind<node_t> uf;// used to identify the two partitions of nodes
    GraphCut(size_t _cut_size, const UnionFind<node_t> &_uf) : cut_size(_cut_size), uf(_uf) {}

    bool operator<(GraphCut const &other) const {
        return cut_size < other.cut_size;
    }

    auto get_partitions() const {
        std::vector<node_t> P;
        std::vector<node_t> Q;
        P.reserve(uf.subsets[0].size);
        Q.reserve(size(uf.subsets) - size(P));

        for (node_t i = 0; i < size(uf.subsets); ++i) {
            uf.subsets[i].id == uf.subsets[0].id ? P.push_back(i) : Q.push_back(i);
        }

        return std::array{P, Q};
    }
};

/* Karger's contraction algorithm in O(n + mα(n)) using n Union-Find data structure to keep track
of merged vertices. The graph is assumed to be connected and nodes indexed between 0 and n-1. Repeat
this function C(n,2)*log(n) = n*(n-1)/2*log(n) for high probability of obtaining the minimum global
cut. The graph isn't per se modified, only its vector of edges is shuffled. */
template<typename node_t>
auto karger_union_find(EdgesVectorGraph<node_t> &graph) -> GraphCut<node_t> {
    auto &mt = prng_engine();
    UnionFind uf{graph.n_vertices};

    auto start = begin(graph.edges);
    for (int m = size(graph.edges) - 1; uf.n_subsets != 2; ++start, --m) {
        iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
        uf.merge(start->tail, start->head);
    }

    return {
            static_cast<size_t>(count_if(
                    start, end(graph.edges), [&](auto e) {
                        return !uf.connected(e.tail, e.head);
                    })),
            std::move(uf)
    };
}

/* A data structure to hold an intermediate contracted graph state. The Union-Find structure
is used to keep track of the merged nodes. */
template<typename node_t>
struct ContractedGraph : EdgesVectorGraph<node_t> {
    UnionFind<node_t> uf;
};

/* Karger-Stein's contraction recursive algorithm. Instead of using a straightforward recursion, we
keep the intermediate graphs to contract in a stack. Repeat this function log²(n) for high probability
 of obtaining the minimum global cut. */
template<typename node_t>
auto karger_stein_union_find(ContractedGraph<node_t> &input_graph) -> GraphCut<node_t> {
    /* Contracts the given graph until it has n_vertices vertices. The graph isn't per se modified,
    only its vector of edges is shuffled. */
    auto contract = [&mt = prng_engine()](ContractedGraph<node_t> &graph,
                                          node_t n_vertices) -> ContractedGraph<node_t> {
        UnionFind uf{graph.uf};

        auto start = begin(graph.edges);
        for (int m = size(graph.edges) - 1; uf.n_subsets != n_vertices; ++start, --m) {
            iter_swap(start, start + std::uniform_int_distribution<>{0, m}(mt));
            uf.merge(start->tail, start->head);
        }

        decltype(graph.edges) edges;
        edges.reserve(end(graph.edges) - start);
        auto it = copy_if(start, end(graph.edges), begin(edges),
                          [&](auto e) {
                              return !uf.connected(e.tail, e.head);
                          });// remove self-loops
        edges.erase(it, edges.end());

        return ContractedGraph<node_t>{n_vertices, std::move(edges), std::move(uf)};
    };

    /* Returns the cut represented by an intermediate contracted graph. The given graph is supposed
    to have no self-loops and to have two nodes (components). */
    auto cut = [](ContractedGraph<node_t> &&graph) -> GraphCut<node_t> {
        return GraphCut{size(graph.edges), graph.uf};
    };

    if (input_graph.n_vertices <= N_VERTICES_THRESHOLD) {
        return cut(contract(input_graph, 2));
    } else {
        node_t t = 1 + ceil(input_graph.n_vertices * INV_SQRT_2);
        ContractedGraph<node_t> g1 = contract(input_graph, t);
        ContractedGraph<node_t> g2 = contract(input_graph, t);
        return std::min(karger_stein_union_find(g1), karger_stein_union_find(g2));
    }
}


Overwriting recursive_karger.hpp


In [12]:
%%writefile recursive_karger_stein_bm.cpp
#include <benchmark/benchmark.h>

#include "recursive_karger.hpp"

static void BM_Karger_Stein_Recursive(benchmark::State &state) {
    auto n_vertices = static_cast<int>(state.range(0));
    size_t n_edges = 0;

    for (auto _: state) {
        state.PauseTiming();

        EdgesVectorGraph<uint32_t> graph = generateRandomConnectedGraph<uint32_t>(n_vertices);
        ContractedGraph<uint32_t> c_graph{graph.n_vertices, std::move(graph.edges),
                                          UnionFind<uint32_t>{graph.n_vertices}};
        n_edges = c_graph.edges.size();

        state.ResumeTiming();

        benchmark::DoNotOptimize(karger_stein_union_find<uint32_t>(c_graph));
    }
    state.counters["|V|"] = n_vertices;
    state.counters["|E|"] = static_cast<int>(n_edges);

    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_Karger_Stein_Recursive)
        ->RangeMultiplier(2)
        ->Range(1U << 2U, 1U << 10U)
        ->Complexity();

BENCHMARK_MAIN();


Overwriting recursive_karger_stein_bm.cpp


## Building code

In [13]:
!clang++-10 original_karger_bm.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o original_karger_bm -g

In [14]:
!clang++-10 karger_bm.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o karger_bm -g

In [15]:
!clang++-10 original_karger_stein_bm.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o original_karger_stein_bm -g

In [16]:
!clang++-10 karger_stein_bm.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o karger_stein_bm -g

In [17]:
!clang++-10 recursive_karger_stein_bm.cpp -O2 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o recursive_karger_stein_bm -g

## Running benchmarks

In [18]:
!./original_karger_bm --benchmark_counters_tabular=true

2021-10-18T20:58:15+00:00
Running ./original_karger_bm
Run on (2 X 2200.15 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.93, 0.49, 0.40
----------------------------------------------------------------------------------------
Benchmark                        Time             CPU   Iterations        |E|        |V|
----------------------------------------------------------------------------------------
BM_Original_Karger/4           705 ns          690 ns       985620          3          4
BM_Original_Karger/8          1073 ns         1050 ns       658704         14          8
BM_Original_Karger/16         1999 ns         1963 ns       355594         60         16
BM_Original_Karger/32         4417 ns         4316 ns       162576        248         32
BM_Original_Karger/64        11042 ns        10862 ns        65051       1008         64
BM_Original_Karger/128       31631 ns        31302 ns 

In [19]:
!./karger_bm --benchmark_counters_tabular=true

2021-10-18T21:04:22+00:00
Running ./karger_bm
Run on (2 X 2200.15 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 1.01, 0.92, 0.65
-------------------------------------------------------------------------------
Benchmark               Time             CPU   Iterations        |E|        |V|
-------------------------------------------------------------------------------
BM_Karger/4           683 ns          681 ns      1018603          3          4
BM_Karger/8          1063 ns         1054 ns       659776         14          8
BM_Karger/16         2047 ns         2010 ns       347860         60         16
BM_Karger/32         4492 ns         4399 ns       159451        248         32
BM_Karger/64        10769 ns        10594 ns        65878       1008         64
BM_Karger/128       29078 ns        28612 ns        24449     4.064k        128
BM_Karger/256       88260 ns        86721 ns         80

In [24]:
!./original_karger_stein_bm --benchmark_counters_tabular=true

2021-10-18T21:23:28+00:00
Running ./original_karger_stein_bm
Run on (2 X 2200.15 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.35, 0.26, 0.47
----------------------------------------------------------------------------------------------
Benchmark                              Time             CPU   Iterations        |E|        |V|
----------------------------------------------------------------------------------------------
BM_Original_Karger_Stein/4           885 ns          864 ns       814660          3          4
BM_Original_Karger_Stein/8          4288 ns         4268 ns       163022         14          8
BM_Original_Karger_Stein/16       140823 ns       140824 ns         4832         60         16
BM_Original_Karger_Stein/32      1069101 ns      1068869 ns          649        248         32
BM_Original_Karger_Stein/64     13506470 ns     13502111 ns           53       1008         64


In [21]:
!./karger_stein_bm --benchmark_counters_tabular=true

2021-10-18T21:11:47+00:00
Running ./karger_stein_bm
Run on (2 X 2200.15 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 1.00, 1.00, 0.81
-------------------------------------------------------------------------------------
Benchmark                     Time             CPU   Iterations        |E|        |V|
-------------------------------------------------------------------------------------
BM_Karger_Stein/4           876 ns          859 ns       783624          3          4
BM_Karger_Stein/8          4124 ns         4107 ns       168740         14          8
BM_Karger_Stein/16       132610 ns       132617 ns         5346         60         16
BM_Karger_Stein/32       997710 ns       997706 ns          709        248         32
BM_Karger_Stein/64     12211781 ns     12208397 ns           56       1008         64
BM_Karger_Stein/128    78349401 ns     78334004 ns            9     4.064k       

In [22]:
!./recursive_karger_stein_bm --benchmark_counters_tabular=true

2021-10-18T21:12:29+00:00
Running ./recursive_karger_stein_bm
Run on (2 X 2200.15 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 1.00, 1.00, 0.82
-----------------------------------------------------------------------------------------------
Benchmark                               Time             CPU   Iterations        |E|        |V|
-----------------------------------------------------------------------------------------------
BM_Karger_Stein_Recursive/4           695 ns          688 ns      1001244          3          4
BM_Karger_Stein_Recursive/8          3711 ns         3696 ns       189365         14          8
BM_Karger_Stein_Recursive/16       133315 ns       133273 ns         5285         60         16
BM_Karger_Stein_Recursive/32      1007922 ns      1007066 ns          691        248         32
BM_Karger_Stein_Recursive/64     12635686 ns     12632446 ns           55       1008   